In [61]:
import numpy as np
import pandas as pd
from pulp import *

In [62]:
inbound = pd.read_excel("Advanced NW Modelling/inbound.xlsx")
inbound.set_index("DCS", inplace=True)
inbound

,plant 1,plant 2,plant 3
DCS,,,
DC_1,22,71,79
DC_2,64,27,26
DC_3,72,36,56
DC_4,50,75,56
DC_5,32,33,21
DC_6,63,34,65
DC_7,37,52,46
DC_8,33,80,42
DC_9,80,54,76


In [63]:
outbound = pd.read_excel("Advanced NW Modelling/outbound.xlsx")
outbound.set_index("DCS", inplace=True)
outbound

,Shop 1,shop 2,Shop 3,Shop 4,Shop 5,Shop 6,Shop 7,Shop 8,Shop 9,Shop 10,Shop 11,Shop 12
DCS,,,,,,,,,,,,
DC_1,98,67,103,117,94,116,75,104,103,71,71,41
DC_2,109,85,114,50,64,53,46,62,50,98,44,43
DC_3,57,116,57,119,53,63,106,74,75,62,43,118
DC_4,113,57,92,80,56,77,55,80,92,70,67,52
DC_5,84,73,91,77,98,85,46,58,112,67,89,102
DC_6,101,69,85,92,43,96,64,73,55,111,49,46
DC_7,109,112,55,56,54,82,60,63,111,48,75,112
DC_8,118,116,56,117,55,54,71,108,93,40,115,66
DC_9,92,111,51,59,58,115,70,86,109,116,70,68


In [64]:
demand = pd.read_excel("Advanced NW Modelling/demand.xlsx")
demand.set_index("shop", inplace=True)
demand

,Demand
shop,
Shop 1,881
shop 2,806
Shop 3,704
Shop 4,767
Shop 5,575
Shop 6,741
Shop 7,593
Shop 8,708
Shop 9,800


In [65]:
plant_costs = pd.read_excel("Advanced NW Modelling/plant+costs.xlsx")
plant_costs.set_index("plant", inplace=True)
plant_costs

,Var,capacity,fixed cost
plant,,,
plant 1,18,3000,40000
plant 2,22,5000,60000
plant 3,32,4000,50000


In [66]:
dc_costs = pd.read_excel("Advanced NW Modelling/DC_costs.xlsx")
dc_costs.set_index("DCS", inplace=True)
dc_costs['Capacity'] = 10000
dc_costs

,Variable Cost,Fixed Cost,Capacity
DCS,,,
DC_1,12,20000,10000
DC_2,15,22000,10000
DC_3,18,40000,10000
DC_4,45,50000,10000
DC_5,23,60000,10000
DC_6,44,12000,10000
DC_7,32,18000,10000
DC_8,12,70000,10000
DC_9,23,65000,10000


In [67]:
plantsvar = plant_costs.index.to_list()
dcsvar = dc_costs.index.to_list()
shopsvar = demand.index.to_list()

In [68]:
# define dist matrix
# using dictionary comprehension
dist_matrix = {(w,s):1 if outbound.loc[w,s] <= 80 else 0 for w in dcsvar for s in shopsvar}

In [69]:
# creating keys and variables
inbound_keys = [(p, w) for w in dcsvar for p in plantsvar]
outbound_keys = [(w, s) for w in dcsvar for s in shopsvar]

In [70]:
inboundvar = LpVariable.dicts("inboundvar",inbound_keys,lowBound=0,upBound=None,cat="Integer")
outboundvar = LpVariable.dicts("outboundvar",outbound_keys,lowBound=0,upBound=None,cat="Integer")
open_dcvar = LpVariable.dicts("opendc",dcsvar,cat="Binary")
open_plantvar = LpVariable.dicts("openplants",plantsvar,cat="Binary")

In [71]:
# add cost calculations
inboundcost = lpSum(inboundvar[(p,w)]*inbound.loc[w,p] for p in plantsvar for w in dcsvar)
outboundcost = lpSum(outboundvar[(w,s)]*outbound.loc[w,s] for w in dcsvar for s in shopsvar)
productioncost = lpSum(inboundvar[(p,w)]*plant_costs.loc[p,'Var'] for p in plantsvar for w in dcsvar)
openplantcost = lpSum(open_plantvar[p]*plant_costs.loc[p,"fixed cost"] for p in plantsvar)
operatedccost = lpSum(inboundvar[(p,w)]*dc_costs.loc[w,"Variable Cost"] for p in plantsvar for w in dcsvar)
opendccost = lpSum(open_dcvar[w]*dc_costs.loc[w,"Fixed Cost"] for w in dcsvar)

In [72]:
# define the model
model = LpProblem("Extended", LpMinimize)

# objective function
model += inboundcost+outboundcost+productioncost+openplantcost+operatedccost+opendccost

In [73]:
# demand constraints
for s in shopsvar:
    model += lpSum(outboundvar[(w,s)] for w in dcsvar) >= demand.loc[s,"Demand"]

# plant capacity constraints
for p in plantsvar:
    model += lpSum(inboundvar[(p,w)] for w in dcsvar) <= lpSum(plant_costs.loc[p,"capacity"]*open_plantvar[p])

# plant capacity constraints
for w in dcsvar:
    model += lpSum(inboundvar[(p,w)] for p in plantsvar) <= lpSum(dc_costs.loc[w,"Capacity"]*open_dcvar[w])

# flow constraints
for w in dcsvar:
    model += lpSum(inboundvar[(p,w)] for p in plantsvar) == lpSum(outboundvar[(w,s)] for s in shopsvar)

# open nos constraints
model += lpSum(open_dcvar[w] for w in dcsvar) >= 1
model += lpSum(open_dcvar[w] for w in dcsvar) <= 9
model += lpSum(open_plantvar[p] for p in plantsvar) >= 1
model += lpSum(open_plantvar[p] for p in plantsvar) <= 3

# service level constraints
model += lpSum(dist_matrix[(w,s)]*outboundvar[(w,s)] for w in dcsvar for s in shopsvar) / demand['Demand'].sum() >= 0.95

In [74]:
# solving the model
model.solve()

1

In [75]:
value(model.objective)

1256177

In [76]:
# note how the variables are stored in the model and called by .name and .varValue for print
for v in model.variables() :
    print (v.name, "=" ,v.varValue)

inboundvar_('plant_1',_'DC_1') = 2137
inboundvar_('plant_1',_'DC_2') = 0
inboundvar_('plant_1',_'DC_3') = 0
inboundvar_('plant_1',_'DC_4') = 0
inboundvar_('plant_1',_'DC_5') = 0
inboundvar_('plant_1',_'DC_6') = 0
inboundvar_('plant_1',_'DC_7') = 0
inboundvar_('plant_1',_'DC_8') = 0
inboundvar_('plant_1',_'DC_9') = 0
inboundvar_('plant_2',_'DC_1') = 0
inboundvar_('plant_2',_'DC_2') = 3415
inboundvar_('plant_2',_'DC_3') = 1585
inboundvar_('plant_2',_'DC_4') = 0
inboundvar_('plant_2',_'DC_5') = 0
inboundvar_('plant_2',_'DC_6') = 0
inboundvar_('plant_2',_'DC_7') = 0
inboundvar_('plant_2',_'DC_8') = 0
inboundvar_('plant_2',_'DC_9') = 0
inboundvar_('plant_3',_'DC_1') = 0
inboundvar_('plant_3',_'DC_2') = 1392
inboundvar_('plant_3',_'DC_3') = 0
inboundvar_('plant_3',_'DC_4') = 0
inboundvar_('plant_3',_'DC_5') = 0
inboundvar_('plant_3',_'DC_6') = 0
inboundvar_('plant_3',_'DC_7') = 0
inboundvar_('plant_3',_'DC_8') = 0
inboundvar_('plant_3',_'DC_9') = 0
opendc_DC_1 = 1
opendc_DC_2 = 1
opendc_DC_3

In [77]:
len(model.variables())

147

In [78]:
value(model.objective)

1256177

In [79]:
# calculate total demand travelled wrt distance
disttravel = lpSum(dist_matrix[(w,s)]*outboundvar[(w,s)].varValue for w in dcsvar for s in shopsvar)
disttravel

8529

In [80]:
totdemand = demand['Demand'].sum()
totdemand

8529

In [81]:
# calcuate service level
servicelevel = disttravel/totdemand
servicelevel

1.0